In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [12]:
import spacy
import pandas as pd
import numpy as np
import random
import sys
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping

from util import remove_fullspace_and_newline, reweight_distribution, sample

In [4]:
# 歌詞データの読み込み
df = pd.read_csv('./datasets/all_songs.csv')
lyrics = df['Lyric']
lyrics.head()

0    まるで荒れる波涛のように背筋つらぬき　心狂わす出逢いそう　出逢い夢うつつと見紛(みまご)うほ...
1    Don't Don't Don't Stay Good-byeDon't Don't Don...
2    YES♪広い空のようなみんな夢を見てるそして叶えてく輝くこの宇宙(そら)でどこ行(ゆ)こうM...
3    いっしょに歌おういっしょの時代(とき)の中いっしょで行こういっしょな夢を見よういっしょにいよ...
4    アッ！とね　言わせて見たいいっぱい愛があふれるウットリするような世界創ろうガッカリするのまだ...
Name: Lyric, dtype: object

## 前処理

In [5]:
# 前処理(全角スペースや改行を除去)
removed_list = [remove_fullspace_and_newline(lyric) for lyric in lyrics]
removed_list[-2]

'瑠璃色金魚は恋い焦がれる凛と咲き誇る花菖蒲吐き出す空気は泡の模様決してあなたの心に届かないのはなびらひらひらと水面に落ちて震える指先時間が止まるわ目が覚めた余韻の余白外の世界はねえなんて眩しい嘘だとしても罪深過ぎたの眩暈がしても心地いいのはもう求めてるから瑠璃色金魚が見上げるのは凛と佇んだ花菖蒲私あなたのようになれたらもっと上手く微笑えますか灯した明かりは燃えないまま今も青く棚引いている曇った硝子を溶かすほどの秘密もしかして私持ってますか雨は空に落ち愛すれば消えるものと思ってた鏡の世界に逆さまに映った好奇心湧き上がる思いを掬い上げては砂糖漬けにしてまた飲み込むのあなたにいつか味見してほしいと夢を見ながら瑠璃色金魚が知らないのは強く根を張った花菖蒲目の前に見えるもの全てが現実ってことはないのあの時触れてくれた温もり光失くしては枯れていく悲しみで泣く私の涙また毒になってしまう抜け出したい瑠璃色金魚が見上げるのは凛と佇んだ花菖蒲私あなたのようになれたらもっと美しく咲き誇れますか瑠璃色金魚が知らないのは強く根を張った花菖蒲目の前に見えるもの全てが現実ってことはないのあの時触れてくれた温もり光失くしては枯れていく悲しみで泣く私の涙また毒になってしまう抜け出したい私きっと'

In [6]:
# 辞書の準備
# 大量の文書を扱うときはpipeを使うと内部的にバッチ化され，効率化される
nlp = spacy.load('ja_ginza')
docs = list(nlp.pipe(removed_list))

In [7]:
# コーパスの分かち書きの作成
lyrics_wakati = []
for doc in docs:
    lyric_wakati = []
    for word in doc:
        lyric_wakati.append(word.text)  # 各要素はspacy.tokens.token.Tokenなので，textでstrとして抽出
    lyrics_wakati.append(lyric_wakati)
print(lyrics_wakati[-2], sep='\n')      # 整形して瑠璃色金魚と花菖蒲の結果を確認

['瑠璃', '色金', '魚', 'は', '恋い焦がれる', '凛と', '咲き誇る', '花菖蒲', '吐き出す', '空気', 'は', '泡', 'の', '模様', '決して', 'あなた', 'の', '心', 'に', '届か', 'ない', 'の', 'はなびら', 'ひらひら', 'と', '水面', 'に', '落ち', 'て', '震える', '指先', '時間', 'が', '止まる', 'わ', '目', 'が', '覚め', 'た', '余韻', 'の', '余白', '外', 'の', '世界', 'は', 'ねえ', 'なんて', '眩しい', '嘘', 'だ', 'と', 'し', 'て', 'も', '罪深', '過ぎ', 'た', 'の', '眩暈', 'が', 'し', 'て', 'も', '心地', 'いい', 'の', 'は', 'もう', '求め', 'てる', 'から', '瑠璃', '色金', '魚', 'が', '見上げる', 'の', 'は', '凛と', '佇ん', 'だ', '花菖蒲', '私', 'あなた', 'の', 'よう', 'に', 'なれ', 'たら', 'もっと', '上手く', '微笑え', 'ます', 'か', '灯し', 'た', '明かり', 'は', '燃え', 'ない', 'まま', '今', 'も', '青く', '棚引い', 'て', 'いる', '曇っ', 'た', '硝子', 'を', '溶かす', 'ほど', 'の', '秘密', 'もし', 'か', 'し', 'て', '私', '持っ', 'て', 'ます', 'か', '雨', 'は', '空', 'に', '落ち', '愛すれ', 'ば', '消える', 'もの', 'と', '思っ', 'て', 'た', '鏡', 'の', '世界', 'に', '逆さま', 'に', '映っ', 'た', '好奇心', '湧き上がる', '思い', 'を', '掬い上げ', 'て', 'は', '砂糖', '漬け', 'に', 'し', 'て', 'また', '飲み込む', 'の', 'あなた', 'に', 'いつ', 'か', '味見', 'し', 'て', 'ほしい', 'と', '夢', 'を', '見', 'ながら'

In [8]:
# 分かち書きからコーパスの一意な単語のリストを作成
flatten = sum(lyrics_wakati, [])   # 平坦化
print('Corpus length:', len(flatten))

words = sorted(list(set(flatten))) # set(集合)を用いて要素を一意に
print('Unique words:', len(words))

# 歌詞中の単語をリストwordsのインデックスにマッピングするディクショナリ
word_indices = dict((word, words.index(word)) for word in words)

Corpus length: 119916
Unique words: 11362


## 入力と出力(目的値)の作成

In [9]:
maxlen = 5       # 5単語分の歌詞を抽出
step = 3         # 3単語ごとに新しい歌詞をサンプリング
sentences = []   # 抽出された歌詞
next_words = []  # 目的値(次にくる単語)

for i in range(0, len(flatten) - maxlen, step):
    sentences.append(flatten[i: i + maxlen])
    next_words.append(flatten[i + maxlen])

print('Number of sequences:', len(sentences))

Number of sequences: 39971


In [10]:
# ベクトル化(one-hotエンコーディングを適用して文字を二値の配列に格納)
print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1
    
print('shape of x:', x.shape)
print('shape of y:', y.shape)

Vectorization...
shape of x: (39971, 5, 11362)
shape of y: (39971, 11362)


## モデルの構築

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(words))))
model.add(Dense(len(words), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop')

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               5883392   
_________________________________________________________________
dense (Dense)                (None, 11362)             1465698   
Total params: 7,349,090
Trainable params: 7,349,090
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# コールバックの設定
def on_epoch_end(epoch, _):
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print()